In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Configure for presentation
np.set_printoptions(threshold=50, linewidth=50)
import matplotlib as mpl
mpl.rc('font', size=16)

## Table methods

In [2]:
drinks = Table(['Drink', 'Cafe', 'Price']).with_rows([
    ['Milk Tea', 'Panda Tea Lounge', 4],
    ['Espresso', 'Gimme',  2],
    ['Coffee',    'Gimme',  3],
    ['Espresso', "Cafe Gola",   2]
])
drinks

Drink,Cafe,Price
Milk Tea,Panda Tea Lounge,4
Espresso,Gimme,2
Coffee,Gimme,3
Espresso,Cafe Gola,2


In [3]:
drinks.sort("Price", distinct=True)

Drink,Cafe,Price
Espresso,Gimme,2
Coffee,Gimme,3
Milk Tea,Panda Tea Lounge,4


In [4]:
discounts = Table().with_columns(
    'Coupon % off', make_array(25, 50, 5),
    'Location', make_array('Panda Tea Lounge', 'Gimme', 'Gimme')
)
discounts

Coupon % off,Location
25,Panda Tea Lounge
50,Gimme
5,Gimme


**Q:** Create a table with the discounted price of each drink at each cafe that offers discounts, step by step.

Step 1. Join drinks with discounts

In [5]:
a = drinks.join('Cafe', discounts, 'Location')
a

Cafe,Drink,Price,Coupon % off
Gimme,Espresso,2,50
Gimme,Espresso,2,5
Gimme,Coffee,3,50
Gimme,Coffee,3,5
Panda Tea Lounge,Milk Tea,4,25


Step 2. Compute discounts and discard unneccessary columns

In [6]:
a = a.with_column('Discounted Price', a.column(2) * (1 - a.column(3)/100) )
a = a.drop('Price', 'Coupon % off')
a

Cafe,Drink,Discounted Price
Gimme,Espresso,1
Gimme,Espresso,1.9
Gimme,Coffee,1.5
Gimme,Coffee,2.85
Panda Tea Lounge,Milk Tea,3


Step 3. Find the cheapest drink at each cafe.

In [7]:
a.sort('Discounted Price').sort('Cafe', distinct=True) # Correct, Espresso is cheaper

Cafe,Drink,Discounted Price
Gimme,Espresso,1
Panda Tea Lounge,Milk Tea,3


In [8]:
a.group('Cafe', min) # Incorrect answer b/c Coffee is first alphabetically but *not* the cheapest drink at Gimme

Cafe,Drink min,Discounted Price min
Gimme,Coffee,1
Panda Tea Lounge,Milk Tea,3


## Advanced Where

In [9]:
3 > 2

True

In [10]:
1 > 2

False

In [11]:
3 > 2 and 2 > 1

True

In [12]:
3 > 2 and 1 > 2

False

In [13]:
3 > 2 or 1 > 2

True

In [14]:
not 3 > 2

False

In [15]:
np.arange(5) > 2

array([False, False, False,  True,  True])

In [16]:
int(True)

1

In [17]:
int(False)

0

In [18]:
sum(np.arange(5) > 2)

2

## Advanced where for Census

In [19]:
# As of Jan 2017, this census file is online here: 
# http://www2.census.gov/programs-surveys/popest/datasets/2010-2015/national/asrh/nc-est2015-agesex-res.csv

full_census_table = Table.read_table('nc-est2015-agesex-res.csv')
partial = full_census_table.select('SEX', 'AGE', 'POPESTIMATE2010', 'POPESTIMATE2015')
us_pop = partial.relabeled(2, '2010').relabeled(3, '2015')
us_pop

SEX,AGE,2010,2015
0,0,3951330,3978038
0,1,3957888,3968564
0,2,4090862,3966583
0,3,4111920,3974061
0,4,4077551,4020035
0,5,4064653,4018158
0,6,4073013,4019207
0,7,4043046,4148360
0,8,4025604,4167887
0,9,4125415,4133564


In [20]:
us_pop.where('AGE', 70)

SEX,AGE,2010,2015
0,70,2062577,2492490
1,70,954073,1162672
2,70,1108504,1329818


In [21]:
us_pop.where('AGE', 70).where([False, True, True])

SEX,AGE,2010,2015
1,70,954073,1162672
2,70,1108504,1329818


In [22]:
seventy = us_pop.where('AGE', 70)
seventy.column('2010') < 2000000

array([False,  True,  True])

In [23]:
seventy.where(seventy.column('2010') < 2000000)

SEX,AGE,2010,2015
1,70,954073,1162672
2,70,1108504,1329818


In [24]:
us_pop.column('2015') / us_pop.column('2010') > 1.5

array([False, False, False, ..., False, False,
       False])

In [25]:
us_pop.where(us_pop.column('2015') / us_pop.column('2010') > 1.5)

SEX,AGE,2010,2015
1,94,43827,68135
1,95,31736,48015
1,97,14775,23092
1,98,9505,14719
1,99,6104,9577
1,100,9352,15088


## Advanced where for bike sharing
Let's used advanced where to answer questions about bike sharing

In [26]:
citi = Table.read_table('citibike_nyc_201707_500000.csv')
trip = citi.relabeled("start station name", "start")\
           .relabeled("end station name", "end")\
           .select("start", "end")\
           .with_column("duration", citi.column("tripduration")/60)\
           .where("duration", are.below(46))
trip.show(3)

start,end,duration
Metropolitan Ave & Bedford Ave,Bedford Ave & Nassau Ave,6.06667
Lafayette St & E 8 St,2 Ave & E 104 St,35.7
Schermerhorn St & Court St,Court St & Nelson St,5.46667


What was the average duration of all trips?

In [27]:
np.average(trip.column('duration'))

14.242164296152607

What was the average duration of trips that started and ended at the same station?

In [28]:
np.average(trip.where(trip.column('start') == trip.column('end')).column('duration'))

17.073637087146835

What was the average duration of trips that started and ended at different stations?

In [29]:
np.average(trip.where(trip.column('start') != trip.column('end')).column('duration'))

14.166956274195622

## Example Prelim Question

Suppose you are given the following table of data:

In [30]:
trip.show(3)

start,end,duration
Metropolitan Ave & Bedford Ave,Bedford Ave & Nassau Ave,6.06667
Lafayette St & E 8 St,2 Ave & E 104 St,35.7
Schermerhorn St & Court St,Court St & Nelson St,5.46667


What is the name of the station where the most rentals ended? (Assume no ties.)

In [31]:
trip.group('end').sort('count', descending=True).column(0).item(0)

'West St & Chambers St'

For how many stations was the average duration of a trip ending at that station at least 10 minutes?

In [32]:
trip.group('end', np.average).where('duration average', are.above(10)).num_rows

611